# BioData Catalyst Data Release QA
Validation tests in this notebook:
1. [**Patient counts of new studies**](https://basicnotebookinstance-rl0ytn08jb87.notebook.us-east-1.sagemaker.aws/notebooks/biodatacatalyst-pic-sure/access-dashboard-metadata/biodatacatalyst_data_release_QA.ipynb#Validation:-New-study-patient-counts): Patient counts of the new studies from the integration environment are compared to the patient counts in Patient_Count_Per_Consents.csv
2. [**Data dictionary comparison**](https://basicnotebookinstance-rl0ytn08jb87.notebook.us-east-1.sagemaker.aws/notebooks/biodatacatalyst-pic-sure/access-dashboard-metadata/biodatacatalyst_data_release_QA.ipynb#Validation:-Data-dictionary-comparison): Integration and production data dictionaries are compared to ensure complete match

### Prerequisites
- Developer access to the integration enviroment (token)
- Consent value(s) of the new study (or studies) to validate (phs number)
- Knowledge on whether a harmonized study was added

### Install packages

In [1]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git
!{sys.executable} -m pip install -r requirements.txt

  Cloning https://github.com/hms-dbmi/pic-sure-python-client.git to /tmp/pip-req-build-usiuhzif
  Running command git clone --filter=blob:none -q https://github.com/hms-dbmi/pic-sure-python-client.git /tmp/pip-req-build-usiuhzif
  Resolved https://github.com/hms-dbmi/pic-sure-python-client.git to commit aabcc6574eede2dc3de410c6c75f7f77ea18d23c
  Preparing metadata (setup.py) ... done
  Created wheel for PicSureClient: filename=PicSureClient-0.1.0-py2.py3-none-any.whl size=10300 sha256=34389687c3d35515b134ba5ae0940592283368dcde1ce29c1c717fb68b3c7654
  Stored in directory: /tmp/pip-ephem-wheel-cache-zzghzxtm/wheels/31/ef/21/e362bba8de04e0072fafec9f77bd1abdf7e166213d27e98729
Successfully built PicSureClient
  Cloning https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git to /tmp/pip-req-build-gi3zzk9i
  Running command git clone --filter=blob:none -q https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git /tmp/pip-req-build-gi3zzk9i
  Resolved https://github.com/hms-dbmi/pic-su

In [2]:
import json
from pprint import pprint

import pandas as pd
import math

from shutil import copyfile
import PicSureClient
import PicSureBdcAdapter

from utils import get_full_consent_vals, compare_datadict_indices, compare_datadicts, get_topmed_and_harmonized_consents

### Connect to PIC-SURE
Be sure to use the **developer token** from the **integration environment**. It is necessary to have access to all studies to validate the counts.

In [3]:
#integration_PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
#int_resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
#int_resource_id = "70c837be-5ffc-11eb-ae93-0242ac130002" # Open access
#integration_token_file = "token.txt"

In [4]:
#with open(integration_token_file, "r") as f:
#    my_int_token = f.read()

In [7]:
#int_client = PicSureClient.Client()
#int_connection = int_client.connect(integration_PICSURE_network_URL, my_int_token, True)
#int_adapter = PicSureBdcAdapter.Adapter(int_connection, my_int_token)
#int_resource = int_adapter.useResource(int_resource_id)



+=========================================================================================+
|        [ WARNING ] you are specifying that you WANT to allow self-signed SSL            |
|        certificates to be acceptable for connections.  This may be useful for           |
|        working in a development environment or on systems that host public              |
|        data.  BEST SECURITY PRACTICES ARE THAT IF YOU ARE WORKING WITH SENSITIVE        |
|        DATA THEN ALL SSL CERTS BY THOSE EVIRONMENTS SHOULD NOT BE SELF-SIGNED.          |
+=========================================================================================+

+--------------------------------------+------------------------------------------------------+
|  Resource UUID                       |  Resource Name                                       |
+--------------------------------------+------------------------------------------------------+
| ERROR:                               |                         

AttributeError: 'Connection' object has no attribute 'decode'

In [62]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id = "70c837be-5ffc-11eb-ae93-0242ac130002" # Be sure to use Open Access resource id
token_file = "int_token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)



+=========================================================================================+
|        [ WARNING ] you are specifying that you WANT to allow self-signed SSL            |
|        certificates to be acceptable for connections.  This may be useful for           |
|        working in a development environment or on systems that host public              |
|        data.  BEST SECURITY PRACTICES ARE THAT IF YOU ARE WORKING WITH SENSITIVE        |
|        DATA THEN ALL SSL CERTS BY THOSE EVIRONMENTS SHOULD NOT BE SELF-SIGNED.          |
+=========================================================================================+

+--------------------------------------+------------------------------------------------------+
|  Resource UUID                       |  Resource Name                                       |
+--------------------------------------+------------------------------------------------------+
| 70c837be-5ffc-11eb-ae93-0242ac130002 | open-hpds               

In [63]:
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)
dictionary = bdc.useDictionary().dictionary()
resource = bdc.useResource(resource_id)

+--------------------------------------+------------------------------------------------------+
|  Resource UUID                       |  Resource Name                                       |
+--------------------------------------+------------------------------------------------------+
| 70c837be-5ffc-11eb-ae93-0242ac130002 | open-hpds                                            |
| ca0ad4a9-130a-3a8a-ae00-e35b07f1108b | visualization                                        |
| 02e23f52-f354-4e8b-992c-d37c8b9ba140 | auth-hpds                                            |
| 36363664-6231-6134-2d38-6538652d3131 | dictionary                                           |
+--------------------------------------+------------------------------------------------------+


## Validation: New study patient counts
The purpose of this section is to validate the patient counts for newly ingested studies.

### Specify the new study (or studies) to be tested
To validate the new studies ingested, specify the phs numbers in the cell below without the consent group. 
For example, if the study of interest is the AMISH study, list `phs000956` below (*not* `phs000956.c1`).

In [64]:
#to_validate = ['list', 'phs_numbers', 'here']
to_validate = ['phs002694', #ACTIV4a,
               'phs002710', #ACTIV4b
               'phs002752' #C3PO
              ]

to_validate_topmed, to_validate_harmonized = get_topmed_and_harmonized_consents(to_validate)

### Get patient count file from S3 bucket
This notebook uses `Patient_Count_Per_Consents.csv` from the S3 bucket as the reference file. First we need to copy this file over to this directory.

In [65]:
src = '/home/ec2-user/SageMaker/studies/ALL-avillach-73-bdcatalyst-etl/general/completed/Patient_Count_Per_Consents.csv'
dst = '/home/ec2-user/SageMaker/biodatacatalyst-pic-sure/access-dashboard-metadata/Patient_Count_Per_Consents.csv'
#copyfile(src, dst)

In [66]:
# Load S3 file as a dataframe in the Jupyter Notebook
patient_ref_file = pd.read_csv('Patient_Count_Per_Consents.csv', header=None, names=['consent', 'patient_count'])
patient_ref_file

,consent,patient_count
0,phs001001.c1,933
1,phs001001.c2,92
2,phs001345.c1,1854
3,phs002694.c1,1083
4,phs000956.c2,1123
...,...,...
114,phs000287.c4,8
115,phs001412.c2,3
116,phs001024.c1,128
117,phs001412.c1,402


In [67]:
# Extract the consent groups based on the user-identified phs values
full_phs = get_full_consent_vals(to_validate, patient_ref_file)
full_phs

['phs002694.c1', 'phs002710.c1', 'phs002752.c1']

### Get patient count for the specified consent groups
Now the consent groups will be used to find the patient counts currently in the integration environment.

In [68]:
# Start a new query and initialize output dictionary
patient_count_query = resource.query()
output = {}


for consentGroup in full_phs:
    print(consentGroup)
    patient_count_query.filter().delete("\\_consents\\") # Delete all consents
    patient_count_query.filter().add("\\_consents\\", consentGroup) # Add back consent group of interest
    patient_count_query.show()
    patient_count = patient_count_query.getCount() # Get patient count
    output[consentGroup] = patient_count # Add to output

phs002694.c1
Deleted key: \_consents\
.__________[ Query.select()  Settings ]_____________________________________________________________________________________________________________________
| _key__________________________________________________________________________________________________________________________
|  \\_Topmed Study Accession with Subject ID\\                                                                                      |
|  \\_Parent Study Accession with Subject ID\\                                                                                      |
.__________[ Query.crosscounts()  has NO SELECTIONS ]_______________________________________________________________________________________________________
.__________[ Query.require() has NO SELECTIONS ]____________________________________________________________________________________________________________
.__________[ Query.anyof()  has NO SELECTIONS ]_______________________________________________

In [69]:
output

{'phs002694.c1': 1083, 'phs002710.c1': 657, 'phs002752.c1': 511}

### Compare the values to the reference file
Finally we will compare the values from the reference file to the counts in the integration environment.

In [70]:
for consent_val in output.keys():
    ref_count = int(patient_ref_file[patient_ref_file['consent'] == consent_val]['patient_count']) # Count from reference file
    integration_count = output[consent_val] # Count from integration environment
    # Display result message
    if ref_count == integration_count:
        print(consent_val, "passes validation")
    else:
        print('***DID NOT PASS VALIDATION:', consent_val)
        print('Expected count from Patient_Count_Per_Consents.csv:\t', ref_count)
        print('Count retrieved from integration environment:\t', integration_count)

phs002694.c1 passes validation
phs002710.c1 passes validation
phs002752.c1 passes validation


## Validation: Data dictionary comparison
The purpose of this section is to compare the data dictionaries of the production and integration environments. These data dictionaries should be identical besides the studies that are being loaded and/or updated.

### Establish connection to production environment

In [72]:
prod_PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure" 
prod_resource_id = "70c837be-5ffc-11eb-ae93-0242ac130002" # Be sure to use Open Access resource id
prod_token_file = "prod_token.txt"

with open(prod_token_file, "r") as f:
    prod_token = f.read()
    
prod_client = PicSureClient.Client()
prod_connection = prod_client.connect(prod_PICSURE_network_URL, prod_token, True)



+=========================================================================================+
|        [ WARNING ] you are specifying that you WANT to allow self-signed SSL            |
|        certificates to be acceptable for connections.  This may be useful for           |
|        working in a development environment or on systems that host public              |
|        data.  BEST SECURITY PRACTICES ARE THAT IF YOU ARE WORKING WITH SENSITIVE        |
|        DATA THEN ALL SSL CERTS BY THOSE EVIRONMENTS SHOULD NOT BE SELF-SIGNED.          |
+=========================================================================================+

+--------------------------------------+------------------------------------------------------+
|  Resource UUID                       |  Resource Name                                       |
+--------------------------------------+------------------------------------------------------+
| 70c837be-5ffc-11eb-ae93-0242ac130002 | open-hpds               

In [73]:
prod_bdc = PicSureBdcAdapter.Adapter(prod_PICSURE_network_URL, prod_token)
#dictionary = bdc.useDictionary().dictionary()
prod_resource = prod_bdc.useResource(prod_resource_id)

+--------------------------------------+------------------------------------------------------+
|  Resource UUID                       |  Resource Name                                       |
+--------------------------------------+------------------------------------------------------+
| 70c837be-5ffc-11eb-ae93-0242ac130002 | open-hpds                                            |
| ca0ad4a9-130a-3a8a-ae00-e35b07f1108b | visualization                                        |
| 02e23f52-f354-4e8b-992c-d37c8b9ba140 | auth-hpds                                            |
| 36363664-6231-6134-2d38-6538652d3131 | dictionary                                           |
+--------------------------------------+------------------------------------------------------+


In [55]:
#production_PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
#prod_resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
#prod_resource_id = "70c837be-5ffc-11eb-ae93-0242ac130002" # Open access
#production_token_file = "prod_token.txt"

In [ ]:
#with open(production_token_file, "r") as f:
#    my_prod_token = f.read()

In [57]:
#prod_client = PicSureClient.Client()
#prod_connection = prod_client.connect(production_PICSURE_network_URL, my_token, True)
#prod_adapter = PicSureBdcAdapter.Adapter(prod_connection)
#prod_resource = prod_adapter.useResource(prod_resource_id)



+=========================================================================================+
|        [ WARNING ] you are specifying that you WANT to allow self-signed SSL            |
|        certificates to be acceptable for connections.  This may be useful for           |
|        working in a development environment or on systems that host public              |
|        data.  BEST SECURITY PRACTICES ARE THAT IF YOU ARE WORKING WITH SENSITIVE        |
|        DATA THEN ALL SSL CERTS BY THOSE EVIRONMENTS SHOULD NOT BE SELF-SIGNED.          |
+=========================================================================================+

+--------------------------------------+------------------------------------------------------+
|  Resource UUID                       |  Resource Name                                       |
+--------------------------------------+------------------------------------------------------+
| 70c837be-5ffc-11eb-ae93-0242ac130002 | open-hpds               

TypeError: __init__() missing 1 required positional argument: 'token'

### Load data dictionaries
Next we will load the data dictionaries from the integration and production environments as dataframes. These will be used to compare the environments.

In [74]:
#int_dictionary = resource.dictionary()
#prod_dictionary = prod_resource.dictionary()

In [78]:
int_dictionary = bdc.useDictionary().dictionary()
prod_dictionary = prod_bdc.useDictionary().dictionary()

In [81]:
integ = int_dictionary.find().dataframe()

In [82]:
prod = prod_dictionary.find().dataframe()

### Compare dictionaries
The following comparisons will be made between the dictionaries:
1. Find concept paths that exist in integration, but not production
2. Find concept paths that exist in production, but not integration
3. Identify differences in the dataframe between integration and production

The first comparisons use the concept paths, which we will extract and compare now.

In [83]:
first_comparison = compare_datadict_indices(integ, prod, 1, to_validate)

All concepts in production are in integration


In [84]:
second_comparison = compare_datadict_indices(prod, integ, 2)

The following concepts are in production but not in integration :
140333
140334
140335
140336
140337
140338
140339
140340
140341
140342
140343
140344
140345
140346
140347
140348
140349
140350
140351
140352
140353
140354
140355
140356
140357
140358
140359
140360
140361
140362
140363
140364
140365
140366
140367
140368
140369
140370
140371
140372
140373
140374
140375
140376
140377
140378
140379
140380
140381
140382
140383


The third comparison compares the data in the data dictionary. The following function iterates through each row of the data dictionary and compares the integration and production results.

In [85]:
harmonized = get_full_consent_vals(to_validate_harmonized, patient_ref_file)
topmed = get_full_consent_vals(to_validate_topmed, patient_ref_file)

In [96]:
integration=list(integ.index)
for integ_key in integration:
    print(integ_key)
    phs in integ_key for phs in to_validate
    
#phs in integ_key for phs in to_validate

SyntaxError: invalid syntax (<ipython-input-96-0bef2f1168d0>, line 4)

In [86]:
to_check = compare_datadicts(integ, prod, to_validate, full_phs, harmonized, topmed, patient_ref_file)

Initializing variables and getting counts...
Comparing data dictionaries...


TypeError: argument of type 'int' is not iterable

In [ ]:
to_check

In [ ]:
# For next QA process:
# Run counts for each value under _studies_consents - compare between Integration and Production

# Do patient counts for the following, if there is a change, be able to explain:
# All _studies have values for _studies_consents and _consents (_parent_consents, _topmed_consents)
# All top-level paths for studies have values in _consents and _studies_consents and _studies (_parent_consents, _topmed_consents)



# Create a table for each environment
# Compare the tables